Step 1: Removing incomplete sequences
Before the log data can be used to train machine learning models, incomplete sequences need to be removed. Each sequence should start with a log line of an alarm with lower severity (anomaly, minor, major) and finishes with a critical alarm.

In [ ]:
# Finds the first crticial alarm of the log
def find_first_critical(current_site_csv):
    index = 0
    for item in current_site_csv:
        if (item['GRAVITE'] == 'CRITIQUE'):
            return index
        index = index + 1
    return 0

In [ ]:
# Finds the last critical alarm of the log
def find_last_critical(current_site_csv):
    index = 0
    last_index = 0
    for item in current_site_csv:
        if (item['GRAVITE'] == 'CRITIQUE'):
            last_index = index
        index = index + 1
    return last_index

In [ ]:
# Checks, if severity is critical
def is_severity_critical(item):
    if item['GRAVITE'] == 'CRITIQUE':
        return True
    else:
        return False

In [ ]:
import pandas as pd
import csv

for i in range(1,26):
    df = pd.read_csv(str(i) + ".csv", delimiter=';') # Read raw data
    final_csv = []
    current_site_csv = []

    for index, row in df.iterrows():
        if row['ID_SITE_LAFON'] == current_site:
            current_site_csv.append(row)
        else:
            # Delete incomplete sequence until first critical
            first_critical = find_first_critical(current_site_csv)
            del current_site_csv[:first_critical+1]

            # Delete last incomplete sequence from last critical on
            last_critical = find_last_critical(current_site_csv)
            if not is_severity_critical(current_site_csv[-1]):
                del current_site_csv[last_critical+1:]

            for item in current_site_csv:
                final_csv.append(item)

            current_site_csv = []
            current_site = row['ID_SITE_LAFON']

    # Treat last station of file
    # Delete incomplete sequence until first critical
    first_critical = find_first_critical(current_site_csv)
    del current_site_csv[:first_critical+1]

    # Delete last incomplete sequence from last critical on
    last_critical = find_last_critical(current_site_csv)
    if not is_severity_critical(current_site_csv[-1]):
        del current_site_csv[last_critical+1:]

    for item in current_site_csv:
        final_csv.append(item)

    with open(str(i) + "_incomplete_removed.csv", "w", newline='') as f:
        writer = csv.writer(f, delimiter=',', dialect='excel')
    writer.writerows(final_csv)
